In [1]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

# mpl.rcParams['figure.figsize'] = (6,6)
# mpl.rcParams['figure.dpi'] = 100
# mpl.rcParams["image.origin"] = 'lower'

In [12]:
config = {
    "base_dir":        "/eos/home-d/dmapelli/public/latino/",
    "plot_config":     "Full2018v6s5",
    "cut":             "boos_sig_mjjincl",
    "model_version":   "bdt_v1",
    "model_tag":       "boost_5vars_v0",
    "samples_version": "v10",
    "cols": ['mjj_vbs', 'vbs_0_pt', 'vbs_1_pt', 'deltaeta_vbs', 'deltaphi_vbs', 
          'mjj_vjet', 'vjet_0_pt', 'vjet_1_pt', 'vjet_0_eta', 'vjet_1_eta', 
         'Lepton_pt', 'Lepton_eta', 'Lepton_flavour', 
         'PuppiMET', 
         'Zvjets_0', 'Zlep', 
         'Asym_vbs', 'Asym_vjet', 'A_ww', 
         'Mtw_lep', 'w_lep_pt', 'Mww', 'R_ww', 'R_mw', 
         'Centr_vbs', 'Centr_ww'
         ]
}

```.python
"cut":             "boos_sig_mjjincl",
"cut":             "res_sig_mjjincl",

"model_tag":       "res_5depth_v0",

    "cols": ['mjj_vbs', 'deltaeta_vbs', 
            'mjj_vjet', 
            'Lepton_pt', 'Lepton_eta' ]
    "cols": ['mjj_vbs', 'vbs_0_pt', 'vbs_1_pt', 'deltaeta_vbs', 'deltaphi_vbs', 
          'mjj_vjet', 'vjet_0_pt', 'vjet_1_pt', 'vjet_0_eta', 'vjet_1_eta', 
         'Lepton_pt', 'Lepton_eta', 'Lepton_flavour', 
         ]
    "cols": ['mjj_vbs', 'vbs_0_pt', 'vbs_1_pt', 'deltaeta_vbs', 'deltaphi_vbs', 
          'mjj_vjet', 'vjet_0_pt', 'vjet_1_pt', 'vjet_0_eta', 'vjet_1_eta', 
         'Lepton_pt', 'Lepton_eta', 'Lepton_flavour', 
         'PuppiMET', 
         'Zvjets_0', 'Zlep', 
         'Asym_vbs', 'Asym_vjet', 'A_ww', 
         'Mtw_lep', 'w_lep_pt', 'Mww', 'R_ww', 'R_mw', 
         'Centr_vbs', 'Centr_ww'
         ]
 ```

In [13]:
import os

config_base_dir = os.path.join(config["base_dir"], config["plot_config"])

# create the model directory
model_dir   = os.path.join(config_base_dir, config["cut"] , "models",  config["model_version"])
os.makedirs(model_dir, exist_ok=True)

# load numpy
samples_dir = os.path.join(config_base_dir, config["cut"] , "samples", config["samples_version"])
import pickle
signal = pickle.load(open(os.path.join(samples_dir, "for_training/signal_balanced.pkl"),     "rb"))
bkg    = pickle.load(open(os.path.join(samples_dir, "for_training/background_balanced.pkl"), "rb"))

## Samples preparation for NN

In [16]:
# Machine Leanring libraires 
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier


# Import useful Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import recall_score, roc_auc_score, accuracy_score, confusion_matrix, make_scorer, classification_report, roc_curve, auc, f1_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier 

In [25]:
X_sig = signal[config["cols"]].values
X_bkg = bkg[config["cols"]].values[:X_sig.shape[0]]
Y_sig = np.ones(len(X_sig))
Y_bkg = np.zeros(len(X_bkg))
W_sig = (signal["weight_norm"]).values
W_bkg = (bkg["weight_norm"]).values
Wnn_sig = (signal["weight_"]).values
Wnn_bkg = (bkg["weight_"]).values

In [26]:
X = np.vstack([X_sig, X_bkg])
Y = np.hstack([Y_sig, Y_bkg])
W = np.hstack([W_sig, W_bkg])
Wnn = np.hstack([Wnn_sig, Wnn_bkg])

In [27]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
pickle.dump(scaler, open(f"{model_dir}/scaler_model.pkl", "wb"))

NameError: name 'StandardScaler' is not defined

 # BDT 

In [31]:
i = 0
k_out   = 10    # number of iterations
k_in    = 5     # k-fold cross validation

n_estimators  = [3, 9, 16, 32, 64, 100, 120, 140] # 200 32, 64, 100

train_results = []
test_results  = []
val_results  = []

train_results_acc = []
test_results_acc  = []
val_results_acc = []

acc = np.zeros(k_in)
sens = np.zeros(k_in)
spec = np.zeros(k_in)
roc = np.zeros(k_in)

acc_avg  = np.zeros(k_out)
sens_avg = np.zeros(k_out)
spec_avg = np.zeros(k_out) 
auc_avg = np.zeros(k_out) 

while i<k_out: # set number of iterations

    # Split training and testing, then use training set for the k-fold cross validation
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.10, shuffle=True, stratify = Y)
    
    skf = StratifiedKFold(n_splits=k_in, shuffle = True)
    for train_index, val_index in skf.split(X_train, Y_train):
        
        # 4-folds for traing
        X_tr = X_train[train_index]
        y_tr = Y_train[train_index]  
        
        # 1 fold for validation
        X_val = X_train[val_index]
        y_val = Y_train[val_index]    


        rf = RandomForestClassifier(n_estimators=20, class_weight='balanced_subsample', n_jobs=-1)
        rf.fit(X_tr, y_tr) 

        train_pred = rf.predict(X_tr)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_tr, train_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        train_acc = accuracy_score(y_tr, train_pred)                
        train_results.append(roc_auc)
        train_results_acc.append(train_acc)
        
        print(f"Train AUC {roc_auc}, train acc {train_acc}")

        val_pred = rf.predict(X_val)
        false_positive_rate, true_positive_rate, thresholds = roc_curve(y_val, val_pred)
        roc_auc = auc(false_positive_rate, true_positive_rate)
        val_acc = accuracy_score(y_val, val_pred) 
        val_results.append(roc_auc)
        val_results_acc.append(val_acc)      
        
        print(f"Val AUC {roc_auc}, val acc {val_acc}")
                  
    i +=1

Train AUC 0.9972569338616275, train acc 0.9972569338616275
Val AUC 0.7062408581179912, val acc 0.7062408581179912
Train AUC 0.9970740627857362, train acc 0.9970740627857361
Val AUC 0.7039249146757679, val acc 0.7039249146757679
Train AUC 0.9970437644764111, train acc 0.9970437644764111
Val AUC 0.7106803218727139, val acc 0.710680321872714
Train AUC 0.9972571010605876, train acc 0.9972571010605876
Val AUC 0.7092172640819313, val acc 0.7092172640819312
Train AUC 0.9967694745824699, train acc 0.9967694745824698
Val AUC 0.7040965618141917, val acc 0.7040965618141917
Train AUC 0.9971959768363304, train acc 0.9971959768363304
Val AUC 0.7057532910775233, val acc 0.7057532910775232
Train AUC 0.9966168850960073, train acc 0.9966168850960073
Val AUC 0.7097757191613847, val acc 0.7097757191613847
Train AUC 0.9972875777154699, train acc 0.9972875777154699
Val AUC 0.7058034625701048, val acc 0.7058034625701048
Train AUC 0.9968913812019993, train acc 0.9968913812019993
Val AUC 0.7117776152158011, va

# Evaluation

In [ ]:
# Evalutation
print(">>> Computing AUC...")

from sklearn.metrics import roc_auc_score, roc_curve

pred_test  = model.predict(X_test,  batch_size=2048)
pred_train = model.predict(X_train, batch_size=2048)
auc_w_test  = roc_auc_score(y_test, pred_test,  sample_weight=W_test)
auc_w_train = roc_auc_score(y_train,pred_train, sample_weight=W_train)
fpW_test,  tpW_test,  thW_test  = roc_curve(y_test,  pred_test , sample_weight=W_test)
fpW_train, tpW_train, thW_train = roc_curve(y_train, pred_train, sample_weight=W_train)
#print("AUC score: " + str(auc))

fig, ax1 = plt.subplots(figsize=(7,6), dpi=100)

# ax1.plot(fp, tp, label=f"ROC (AUC={auc:.3f})", color="blue")
ax1.plot(fpW_test,  tpW_test,  label=f"ROC test  (AUC={auc_w_test:.3f})", color="red")
ax1.plot(fpW_train, tpW_train, label=f"ROC train (AUC={auc_w_train:.3f})", color="blue")

ax1.set_xlabel("Bkg contamination", fontsize=18)
ax1.set_ylabel("Signal efficiency", fontsize=18)

#ax1.plot([0,1],[1,1],"b--")
ax1.tick_params("y",labelsize="large")

ax1.grid()

ax1.legend(loc=(0.4, 0.5), fontsize="large")

fig.tight_layout()

In [107]:
#r = model.evaluate_generator(training_generator, steps=1000)
#r = model.evaluate_generator(validation_generator, steps=1000)


In [108]:
#print(model.metrics_names, r)

In [ ]:
fig, ax = plt.subplots(figsize=(7,6), dpi=100)
ax.plot(history.epoch, history.history["val_loss"], label="validation loss")
ax.plot(history.epoch, history.history["loss"], label="training loss")
plt.legend()

In [ ]:
results_test  = model.predict(X_test, batch_size=2048)
fig, ax1 = plt.subplots(figsize=(7,6), dpi=100)
plt.hist(results_test[y_test==0],weights=W_test[y_test==0], bins=25, density=True, label="false test", histtype="step")
plt.hist(results_test[y_test==1],weights=W_test[y_test==1], bins=25, density=True, label="true test", histtype="step")
results_train = model.predict(X_train, batch_size=2048)
plt.hist(results_train[y_train==0],weights=W_train[y_train==0], bins=25, density=True, label="false train", histtype="step")
plt.hist(results_train[y_train==1],weights=W_train[y_train==1], bins=25, density=True, label="true train", histtype="step")
#plt.yscale("log")
plt.legend()

# 
# 

#results_train[y_train==1]
rtest  = [x[0] for x in results_test[y_test==1]]
rtrain = [x[0] for x in results_train[y_train==1]]

from scipy import stats
stats.ks_2samp(rtrain, rtest)

## Evaluation by sample

In [57]:
bkg["y"] = model.predict(scaler.transform(bkg[config["cols"]].values), batch_size=2048)

In [58]:
signal["y"] = model.predict(scaler.transform(signal[config["cols"]].values), batch_size=2048)

In [59]:
wjets = bkg[bkg["sample_name"] == "Wjets"]
top = bkg[bkg["sample_name"] == "top"]
dy = bkg[bkg["sample_name"] == "DY"]

In [ ]:
fig, ax1 = plt.subplots(figsize=(7,6), dpi=100)
plt.hist(bkg[bkg["sample_name"] == "Wjets"]["y"], bins=50, density=True, label="W+jets", histtype="step")
plt.hist(bkg[bkg["sample_name"] == "top"]["y"],   bins=50, density=True, label="top",    histtype="step")
plt.hist(bkg[bkg["sample_name"] == "DY"]["y"],    bins=50, density=True, label="DY",     histtype="step")

plt.hist(signal["y"], bins=50,density=True, label="signal", histtype="step")
#plt.yscale("log")
plt.legend()

In [ ]:
fig, ax1 = plt.subplots(figsize=(7,6), dpi=100)
plt.hist(wjets["y"], weights=wjets.weight_norm, bins=50, density=True, label="W+jets", histtype="step")
plt.hist(top["y"],   weights=top.weight_norm,  bins=50, density=True, label="top",    histtype="step")
plt.hist(dy["y"],    weights=dy.weight_norm,  bins=50, density=True, label="DY",     histtype="step")

plt.hist(signal["y"], weights=signal.weight_norm, bins=50,density=True, label="signal", histtype="step")
#plt.yscale("log")
plt.legend()

In [ ]:
## some useful snippets

## callbacks
# auto_save = ModelCheckpoint("../models/model12_balanced_100_50.hd5", monitor='val_loss', 
#                     verbose=1, save_best_only=True, save_weights_only=False, 
#                     mode='auto', period=5)
#
# early_stop = EarlyStopping(monitor='val_loss', min_delta=0.0001, 
#                             patience=10, verbose=1)






## In case you do not want to use the generators
# history = model.fit(
#             X_train,y_train, 
#             sample_weight=W_train, 
#             epochs=50,
#             validation_data = (X_val, y_val, W_val),
#             batch_size=1024,
#             shuffle=True 
#             )